In [ ]:
# Version v04-08
# Import all libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt # ploting the data
import seaborn as sns # ploting the data
import math # calculation
import sys
!{sys.executable} -m pip install scipy
from scipy import stats
from scipy.stats import zscore
import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import sklearn.metrics as sm
from sklearn.metrics import r2_score
from datetime import datetime
import time
import dateutil

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Data load 
a=["date","Appliances","T1","RH_1","T2","RH_2","T3","RH_3","T4","RH_4","T5","RH_5","T6","RH_6","T7","RH_7","T8","RH_8","T9","RH_9","T_out","RH_out","Windspeed","Visibility",'Press_mm_hg', 'Tdewpoint', 'lights']
#load the datasets
df = pd.read_csv(r"../input/appliances-energy-prediction/KAG_energydata_complete.csv",usecols=a)

In [ ]:
# Visualize data info
df.info()
df.shape

In [ ]:
df.head()

2. Data Preprosessing
After loading both data set I checked the info of both data set Checked and sum the null values Cheeked the unique values of different attributes Dropped the irrelevant attributes according to evaluation data set Recalculated the null values and then fill the null values Checked the description of data sets and checked the relevant attributes (mean, minimum, maximum, and std ) for regression make a new development data set according to the price is greater than 0 I applied all the unmentioned steps on the both data sets. I checked the correlation on development data set Rationalize the related attributes values for regression analysis For visualization I plotted the different graphs as well

# 2.1 Extracting components from date object

In [ ]:
df["exact_date"]=df['date'].str.split(' ').str[0]

df["hours"]=(df['date'].str.split(':').str[0].str.split(" ").str[1]).astype(str).astype(int)
df["seconds"]=((df['date'].str.split(':').str[1])).astype(str).astype(int).mul(60)

df["days"]=(df['date'].str.split(' ').str[0])
df["days"]=(df['days'].apply(dateutil.parser.parse, dayfirst=True))
df["days_num"]=(df['days'].dt.dayofweek).astype(str).astype(int)
df["days"]=(df['days'].dt.day_name())

In [ ]:
df['date']

In [ ]:
df.info()

In [ ]:
df.describe()

# 2.2 Visualizing Per day consumption
Seasonality
this is used to track seasonality. but we have only 4.25 months data so there were no traces of seasonality

The traces shows that there is varaiation of electric usage on granular terms (like days)

In [ ]:
dates=df["exact_date"].unique()
arranged_day = pd.Categorical(df["exact_date"], categories=dates,ordered=True)
date_series = pd.Series(arranged_day)
table = pd.pivot_table(df,
               values="Appliances",index=date_series,
               aggfunc=[np.sum],fill_value=0)
table.plot(kind="bar",figsize=(20, 7))
plt.show()

# 2.3 Visualizing data 
by Hour Wise Daily Consumption
we also saw a trend in hourly consumption

In [ ]:



days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday","Saturday","Sunday"]
arranged_day = pd.Categorical(df["days"], categories=days,ordered=True)
day_series = pd.Series(arranged_day)
table = pd.pivot_table(df,index=["hours"],
               values="Appliances",columns=day_series,
               aggfunc=[np.sum],fill_value=0)

fig, ax = plt.subplots(figsize=(20, 10))
ax.set_title('Heatmap : Appliances(wh)')

heatmap = ax.pcolor(table)

ax.set_xlabel("Week Days")
ax.set_ylabel("Hours")

plt.colorbar(heatmap)
ax.set_yticks(range(len(table.index)+1))
ax.set_xticks(range(len(table.columns)+1))

plt.xlabel("Week")
plt.ylabel("Hours of Day")
plt.show()



# 2.4 Ploting
This plot shows that there is no such coorelation between energy conuption on weekdays vs off days

In [ ]:
table.plot.box(figsize=(20, 7))

In [ ]:
df.head()

In [ ]:

table=df[["Appliances",'lights', 'T1', 'RH_1',"Press_mm_hg","RH_out","Windspeed","Visibility","Tdewpoint",'hours','days_num']]

In [ ]:
sns.pairplot(table)

## 2.5 Null Values Check and remove 


In [ ]:
df_d=df
df_d.isnull().sum()

## 2.6 Outlier Removal

In [ ]:
df_d.describe()

In [ ]:
df_d['Appliances'].plot(kind="hist",figsize=(20, 7))
plt.show()

In [ ]:
X=df_d[['Appliances']]
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
print("Outlier threshold of Appliances ",IQR)

dfOutlier=X.describe()
outlierSet=set()
for column in dfOutlier.columns:
    Q1 = dfOutlier[column]['25%']
    Q3 = dfOutlier[column]['75%']
    IQR = Q3 - Q1
    outlierDf= ( ((X[column] < (Q1 - 1.5 * IQR)) |(X[column] > (Q3 + 1.5 * IQR))) )
    outlierSet.update(set(outlierDf[outlierDf==True].index))
        

df_d.drop(outlierSet, inplace=True, axis=0)

In [ ]:
#examine the dataset
df_d.describe()

In [ ]:
df_d.columns

In [ ]:
df_d['Appliances'].plot(kind="hist",figsize=(20, 7))
plt.show()

# 2.8 Correlation

In [ ]:
corr_matrix = df_d.corr(method='pearson')
matrix=corr_matrix["Appliances"].sort_values(ascending=False)
print((matrix))

In [ ]:
df_d.head()

In [ ]:
plt.figure(figsize=(20,10))
title = 'Correlation matrix of numerical variables'
sns.heatmap(df.corr(), square=True, cmap='RdYlGn')
plt.title(title)
plt.ioff()

# 3 Split dataset into Train and Test

In [ ]:
df_d.columns

In [ ]:
# Split the dataset
x= df_d[['lights','T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'T6','RH_6', 'T7', 'RH_7','T8', 'RH_8', 'T9',
       'RH_9', 'T_out', 'RH_out', 'Windspeed', 'Visibility', 
       'hours', 'days_num','Tdewpoint','Press_mm_hg']]
X=x.values
y = df_d['Appliances'].values


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2)


# 4. Modeling
Multi liner regression
Lasso
GBM
Random forest regression
SVM
Algorithm for regression. MLR provided the best results. During the data preprocessing I did not get any polynomial or liner relation of any attributes with price. So I chosen the most relevant attributes values for MLR. I run almost MLR and RFR almost 12 times with the different attributes combination and changing the rationalize factors but at the end MLR gave me the best results.

In [ ]:
def result (y_pred_train, y_pred_test,y_train,y_test):
  print("train results")
  print("Mean absolute error =", round(sm.mean_absolute_error(y_train, y_pred_train), 2)) 
  print("Mean squared error =", round(sm.mean_squared_error(y_train, y_pred_train), 2)) 
  print("Median absolute error =", round(sm.median_absolute_error(y_train, y_pred_train), 2)) 
  print("Explain variance score =", round(sm.explained_variance_score(y_train, y_pred_train), 2)) 
  print("R2 score =", round(sm.r2_score(y_train, y_pred_train), 2))


  print("test results")
  print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_pred_test), 2)) 
  print("Mean squared error =", round(sm.mean_squared_error(y_test, y_pred_test), 2)) 
  print("Median absolute error =", round(sm.median_absolute_error(y_test, y_pred_test), 2)) 
  print("Explain variance score =", round(sm.explained_variance_score(y_test, y_pred_test), 2)) 
  print("R2 score =", round(sm.r2_score(y_test, y_pred_test), 2))




# 4.2.1 Plot Training Deviance

In [ ]:
#Plot Training Deviance

# compute test set deviance
test_score=np.zeros((params['n_estimators'],), dtype=np.float64)

for i,y_pred in enumerate(clf.staged_predict(X_test)):
    test_score[i]=clf.loss_(y_test,y_pred)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title('Deviance')

plt.plot(np.arange(params['n_estimators'])+1, model.train_score_,'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators'])+1,test_score,'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

In [ ]:
# Use the forest's predict method on the test data
predictions = clf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

#  4.3 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=500, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
        max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=[True, False],
        oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
regressor.fit(X_train, y_train)

y_pred_train = regressor.predict(X_train)
y_pred_test = regressor.predict(X_test)

rfr_result=result (y_pred_train, y_pred_test,y_train,y_test)
rfr_result

# 4.3.1 Accuracy

In [ ]:
# Use the forest's predict method on the test data
predictions = regressor.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# 4.3.2 Variable Importance

In [ ]:
# Saving feature names for later use
factor_list = list(x.columns)
# Convert to numpy array
factors = np.array(x)
#Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
factor_importances = [(factor, round(importance, 2)) 
    for factor, importance in zip(factor_list, importances)]
# Sort the feature importances by most important first
factor_importances = sorted(factor_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in factor_importances];

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, factor_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); 
plt.title('Variable Importances');

# 4.3.3 ACTUAL VS PREDICTED

In [ ]:
pred= pd.DataFrame({'Actual': np.round(y_test, 0), 
                   'Predicted': np.round(y_pred_test, 0)})
pred